In [1]:
import torch
import xarray as xr
from torch.utils.data import DataLoader
from carboninference.core import ERA5Dataset, ERA5InferenceConfig
from ecoperceiver.components import EcoPerceiverConfig
from ecoperceiver.model import EcoPerceiver
from pathlib import Path

In [2]:
ds_path = "../pipeline/data.nc"
igbp_path = "./igbp.tiff"
ds = ERA5Dataset(ds_path, igbp_path, config=ERA5InferenceConfig(targets=['FCH4', 'NEE']))

/Users/mathiaslarochelle/Developer/Stage_S25/era5_workbench/inference/carboninference/core.py:74: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  nt = self.data.dims["timestamps"]
/Users/mathiaslarochelle/Developer/Stage_S25/era5_workbench/inference/carboninference/core.py:75: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  nlat = self.data.dims["lat"]
/Users/mathiaslarochelle/Developer/Stage_S25/era5_workbench/inference/carboninference/core.py:76: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with 

In [3]:
ds.data

<xarray.Dataset> Size: 12GB
Dimensions:     (timestamps: 48, lat: 721, lon: 1440)
Coordinates:
    timestamps  (timestamps) datetime64[ns] 384B 2010-07-01 ... 2010-07-02T23...
    lat         (lat) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
    lon         (lon) float64 12kB -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
Data variables: (12/33)
    TA          (timestamps, lat, lon) float64 399MB dask.array<chunksize=(48, 128, 128), meta=np.ndarray>
    P           (timestamps, lat, lon) float64 399MB dask.array<chunksize=(48, 128, 128), meta=np.ndarray>
    RH          (timestamps, lat, lon) float64 399MB dask.array<chunksize=(48, 128, 128), meta=np.ndarray>
    VPD         (timestamps, lat, lon) float64 399MB dask.array<chunksize=(48, 128, 128), meta=np.ndarray>
    PA          (timestamps, lat, lon) float64 399MB dask.array<chunksize=(48, 128, 128), meta=np.ndarray>
    CO2         (timestamps, lat, lon) float64 399MB dask.array<chunksize=(48, 128, 128), meta=np.ndarray>
    ...          ...
    LE          (timestamps, lat, lon) float64 399MB dask.array<chunksize=(48, 128, 128), meta=np.ndarray>
    PPFD_IN     (timestamps, lat, lon) float64 399MB dask.array<chunksize=(48, 128, 128), meta=np.ndarray>
    PPFD_OUT    (timestamps, lat, lon) float64 399MB dask.array<chunksize=(48, 128, 128), meta=np.ndarray>
    WTD         (timestamps, lat, lon) float64 399MB dask.array<chunksize=(48, 128, 128), meta=np.ndarray>
    DOY         (timestamps) float64 384B 182.0 182.0 182.0 ... 183.0 183.0
    TOD         (timestamps) float64 384B 0.0 1.0 2.0 3.0 ... 21.0 22.0 23.0

In [4]:
ds.igbp

<xarray.Dataset> Size: 4MB
Dimensions:  (band: 1, y: 721, x: 1440)
Coordinates:
  * y        (y) float64 6kB -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
  * x        (x) float64 12kB -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8
Dimensions without coordinates: band
Data variables:
    igbp     (band, y, x) float32 4MB nan nan nan nan nan ... nan nan nan nan

In [5]:
config = EcoPerceiverConfig(targets=('NEE', 'GPP', 'FCH4'))
model = EcoPerceiver(config)
model.to('mps')
print('model loaded')

model loaded


In [6]:
dl = DataLoader(ds, batch_size=64, shuffle=True, num_workers=8, pin_memory=True, collate_fn=ds.collate_fn)

In [7]:
batch = next(iter(dl))

/opt/anaconda3/envs/ecoperceiver-env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/anaconda3/envs/ecoperceiver-env/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/ecoperceiver-env/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/Users/mathiaslarochelle/Developer/Stage_S25/era5_workbench/inference/carboninference/core.py", line 115, in __getitem__
    vals = self.data.coords[dim].isel(timestamps=slice(t0, t+1)).values
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/ecoperceiver-env/lib/python3.12/site-packages/xarray/core/dataarray.py", line 1529, in isel
    variable = self._variable.isel(indexers, missi

In [ ]:
print(batch.target_values.shape)